# Retrievers

Retreivers go a step further than simple similarity search, while there is a type called `Vector Store backed retreiver` which does exactly as the name implies, there are other - very powerful - types of retreivers, namely: `Multi-Query Retriever`, `Self-Querying Retriever`, and `Parent Document Retriever`. In this notebook, we explore all four of them.

In [ ]:
%pip install --user "ibm-watsonx-ai==1.1.2" | tail -n 1
%pip install --user "langchain==0.2.1" | tail -n 1
%pip install --user "langchain-ibm==0.1.11" | tail -n 1
%pip install --user "langchain-community==0.2.1" | tail -n 1
%pip install --user "chromadb==0.4.24" | tail -n 1
%pip install --user "pypdf==4.3.1" | tail -n 1
%pip install --user "lark==1.1.9" | tail -n 1

First we need to build an LLM, split documents into chunks and finally build an embedding model to process the chunks.

### Building LLM

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models.extensions.langchain import WatsonxLLM

In [ ]:
def llm():
    model_id = 'mistralai/mixtral-8x7b-instruct-v01'
    
    parameters = {
        GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
        GenParams.TEMPERATURE: 0.5, # this randomness or creativity of the model's responses
    }
    
    credentials = {
        "url": "https://us-south.ml.cloud.ibm.com"
    }
    
    
    project_id = "skills-network"
    
    model = ModelInference(
        model_id=model_id,
        params=parameters,
        credentials=credentials,
        project_id=project_id
    )
    
    mixtral_llm = WatsonxLLM(model = model)
    return mixtral_llm

### Splitting Text

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def text_splitter(data, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    chunks = text_splitter.split_documents(data)
    return chunks

### Building Embedding Model

In [ ]:
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

In [ ]:
def watsonx_embedding():
    embed_params = {
        EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
        EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
    }
    
    watsonx_embedding = WatsonxEmbeddings(
        model_id="ibm/slate-125m-english-rtrvr",
        url="https://us-south.ml.cloud.ibm.com",
        project_id="skills-network",
        params=embed_params,
    )
    return watsonx_embedding

Now we are ready to explore the retrievers ...

## Exploring Retrievers

### Vector Store-Backed Retriever